In [1]:
import pandas as pd
pd.options.display.float_format = '$ {:,.2f}'.format

# Reading files and create dataframe
files = ["purchase_data2.json", "purchase_data.json"]
df = pd.concat([pd.read_json(f) for f in files]).reset_index()

In [20]:
# Part to add "NormalizedPrice" columns
from sklearn import preprocessing

x = df['Price'].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x.reshape(-1, 1))
df["NormalizedPrice"] = x_scaled

In [11]:
# Player Count
PlayerCount = len(df["SN"].unique())

# Result
pd.DataFrame({ "Total Players": PlayerCount }, index=[0]) 

,Total Players
0,612


In [12]:
# Purchasing Analysis (Total)
itemCount = len(df["Item ID"].unique())
averagePrice = df["Price"].mean()
purchaseCount = len(df)
totalRevenue = df["Price"].sum()

purchaseAnalysis = pd.DataFrame({
            "Number of Unique Items": itemCount,
            "Average Price": averagePrice,
            "Number of Purchases": purchaseCount,
            "Total Revenue": totalRevenue
             }, index=[0])

# formatting
purchaseAnalysis = purchaseAnalysis[["Number of Unique Items", "Average Price", "Number of Purchases", "Total Revenue"]]

# Result
purchaseAnalysis

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,184,$ 2.93,858,"$ 2,514.43"


In [13]:
# Gender Demographics
genderDemographics = df.groupby("Gender")["SN"].nunique().to_frame(name="Total Count")
genderDemographics["Percentage of Players"] = round(genderDemographics["Total Count"] / PlayerCount * 100, 2) 

# formatting
del genderDemographics.index.name
genderDemographics = genderDemographics.reindex(["Male", "Female", "Other / Non-Disclosed"])
genderDemographics = genderDemographics[["Percentage of Players", "Total Count"]]
genderDemographics["Percentage of Players"] = genderDemographics["Percentage of Players"].map("{:.2f} %".format)

# Result 
genderDemographics

,Percentage of Players,Total Count
Male,81.37 %,498
Female,18.30 %,112
Other / Non-Disclosed,1.47 %,9


In [14]:
# Purchasing Analysis (Gender)
df_gender = df.groupby("Gender") 

PurchaseCount = df_gender.index.count()
AveragePrice = df_gender.Price.mean()
TotalValue = df_gender.Price.sum()
# NormTotal = df_gender.NormalizedPrice.sum()
NormTotal = TotalValue / AveragePrice

# formatting
columns = ["Purchase Count", "Average Price", "Total Purchase Value", "Normalized Totals"]
purchasingAnalysis_gender = pd.concat([PurchaseCount, AveragePrice, TotalValue, NormTotal], axis=1, keys=columns)
purchasingAnalysis_gender = purchasingAnalysis_gender.reindex(["Male", "Female", "Other / Non-Disclosed"])

# Result 
purchasingAnalysis_gender

,Purchase Count,Average Price,Total Purchase Value,Normalized Totals
Gender,,,,
Male,697,$ 2.94,"$ 2,052.28",$ 697.00
Female,149,$ 2.85,$ 424.29,$ 149.00
Other / Non-Disclosed,12,$ 3.15,$ 37.86,$ 12.00


In [15]:
# Age Demographics
bins = [0, 10, 15, 20, 25, 30, 35, 40, 999]
names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']
df["ageCategory"] = pd.cut(df["Age"], bins, labels=names)

ageDemographics = df["ageCategory"].value_counts().to_frame(name="Total Count")
ageDemographics["Percentage of Players"] = round(ageDemographics["Total Count"] / PlayerCount * 100, 2)

# formatting
ageDemographics = ageDemographics.reindex(names)
ageDemographics = ageDemographics[["Percentage of Players", "Total Count"]]
ageDemographics["Percentage of Players"] = ageDemographics["Percentage of Players"].map("{:.2f} %".format)

# Result 
ageDemographics

,Percentage of Players,Total Count
<10,6.05 %,37
10-14,13.40 %,82
15-19,33.33 %,204
20-24,55.23 %,338
25-29,13.07 %,80
30-34,10.62 %,65
35-39,8.01 %,49
40+,0.49 %,3


In [16]:
# Purchasing Analysis (Age)
df_age = df.groupby("ageCategory")

PurchaseCount = df_age.index.count()
AveragePrice = df_age.Price.mean()
TotalValue = df_age.Price.sum()
# NormTotal = df_age.NormalizedPrice.sum()
NormTotal = TotalValue / PurchaseCount

columns = ["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Normalized Totals"]
purchasingAnalysis_age = pd.concat([PurchaseCount, AveragePrice, TotalValue, NormTotal], axis=1, keys=columns)

# formatting
del purchasingAnalysis_age.index.name
purchasingAnalysis_age = purchasingAnalysis_age.reindex(names)

# Result
purchasingAnalysis_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Normalized Totals
<10,37,$ 2.98,$ 110.44,$ 2.98
10-14,82,$ 2.88,$ 236.36,$ 2.88
15-19,204,$ 2.86,$ 583.43,$ 2.86
20-24,338,$ 2.97,"$ 1,003.03",$ 2.97
25-29,80,$ 2.88,$ 230.59,$ 2.88
30-34,65,$ 3.00,$ 194.73,$ 3.00
35-39,49,$ 3.00,$ 147.21,$ 3.00
40+,3,$ 2.88,$ 8.64,$ 2.88


In [17]:
# Top Spenders
df_SN = df.groupby("SN")

top_five_SN = df_SN["Price"].sum().nlargest(5).to_frame().rename(columns={"Price":"Total Purchase Value"}).reset_index()
purchase_count = df_SN.index.count().to_frame().rename(columns={"index":"Purchase Count"}).reset_index()

topSpenders = top_five_SN.merge(purchase_count)
topSpenders["Average Purchase Price"] = topSpenders["Total Purchase Value"] / topSpenders["Purchase Count"]

# formatting
topSpenders.set_index("SN", inplace=True)
topSpenders = topSpenders[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

# Result
topSpenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Undirrala66,5,$ 3.41,$ 17.06
Aerithllora36,4,$ 3.78,$ 15.10
Saedue76,4,$ 3.39,$ 13.56
Sondim43,4,$ 3.25,$ 13.02
Mindimnya67,4,$ 3.18,$ 12.74


In [18]:
# Most Popular Items
df_group_item = df.groupby(["Item ID", "Item Name", "Price"])

most_sold_item = df_group_item.index.count().nlargest(5).to_frame().rename(columns={"index":"Purchase Count"}).reset_index()
value_item = df_group_item.Price.sum().to_frame().rename(columns={"Price":"Total Purchase Value"}).reset_index()

popularItems_sold = most_sold_item.merge(value_item)

# formatting
popularItems_sold.set_index(["Item ID", "Item Name"], inplace=True)
popularItems_sold = popularItems_sold[["Purchase Count", "Price", "Total Purchase Value"]].rename(columns={"Price":"Item Price"})

# Result
popularItems_sold

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
39,"Betrayal, Whisper of Grieving Widows",11,$ 2.35,$ 25.85
84,Arcane Gem,11,$ 2.23,$ 24.53
13,Serenity,9,$ 1.49,$ 13.41
31,Trickster,9,$ 2.07,$ 18.63
34,Retribution Axe,9,$ 4.14,$ 37.26


In [19]:
# Most Profitable Items
df_group_item = df.groupby(["Item ID", "Item Name", "Price"])

sold_item = df_group_item.index.count().to_frame().rename(columns={"index":"Purchase Count"}).reset_index()
most_value_item = df_group_item.Price.sum().nlargest(5).to_frame().rename(columns={"Price":"Total Purchase Value"}).reset_index()

popularItems_profit = most_value_item.merge(sold_item)

# formatting 
popularItems_profit.set_index(["Item ID", "Item Name"], inplace=True)
popularItems_profit = popularItems_profit[["Purchase Count", "Price", "Total Purchase Value"]].rename(columns={"Price":"Item Price"})

# Result
popularItems_profit

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
34,Retribution Axe,9,$ 4.14,$ 37.26
115,Spectral Diamond Doomblade,7,$ 4.25,$ 29.75
32,Orenmir,6,$ 4.95,$ 29.70
103,Singed Scalpel,6,$ 4.87,$ 29.22
107,"Splitter, Foe Of Subtlety",8,$ 3.61,$ 28.88
